In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_seq = pd.read_csv('smi.csv', header=None)

In [3]:
df_seq.head()

,0
0,smiles
1,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
2,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
3,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
4,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...


In [4]:
len(df_seq)

8289

In [5]:
df_seq[0].values[1]

'CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O'

In [6]:
freq_codes = []

In [7]:
import os
import sys
import inspect
import codecs
import re
import copy
import argparse
import warnings
from collections import defaultdict, Counter


In [8]:
def get_vocabulary(fobj, is_dict=False):
#     Read text and return dictionary that encodes vocabulary
    vocab = Counter()
    for i, line in enumerate(fobj):
        if is_dict:
            try:
                word, count = line.strip('\r\n ').split(' ')
            except:
                print('Failed reading vocabulary file at line {0}: {1}'.format(i, line))
                sys.exit(1)
            vocab[word] += int(count)
        else:
            for word in line.strip('\r\n ').split(' '):
                if word:
                    vocab[word] += 1
        
    
    vocab = Counter()
    for word in fobj:
        vocab[word] += 1
    return vocab

In [9]:
def update_pair_statistics(pair, changed, stats, indices):
    """Minimally update the indices and frequency of symbol pairs
    if we merge a pair of symbols, only pairs that overlap with occurrences
    of this pair are affected, and need to be updated.
    """
    stats[pair] = 0
    indices[pair] = defaultdict(int)
    first, second = pair
    new_pair = first+second
    for j, word, old_word, freq in changed:

        # find all instances of pair, and update frequency/indices around it
        i = 0
        while True:
            # find first symbol
            try:
                i = old_word.index(first, i)
            except ValueError:
                break
            # if first symbol is followed by second symbol, we've found an occurrence of pair (old_word[i:i+2])
            if i < len(old_word)-1 and old_word[i+1] == second:
                # assuming a symbol sequence "A B C", if "B C" is merged, reduce the frequency of "A B"
                if i:
                    prev = old_word[i-1:i+1]
                    stats[prev] -= freq
                    indices[prev][j] -= 1
                if i < len(old_word)-2:
                    # assuming a symbol sequence "A B C B", if "B C" is merged, reduce the frequency of "C B".
                    # however, skip this if the sequence is A B C B C, because the frequency of "C B" will be reduced by the previous code block
                    if old_word[i+2] != first or i >= len(old_word)-3 or old_word[i+3] != second:
                        nex = old_word[i+1:i+3]
                        stats[nex] -= freq
                        indices[nex][j] -= 1
                i += 2
            else:
                i += 1

        i = 0
        while True:
            try:
                # find new pair
                i = word.index(new_pair, i)
            except ValueError:
                break
            # assuming a symbol sequence "A BC D", if "B C" is merged, increase the frequency of "A BC"
            if i:
                prev = word[i-1:i+1]
                stats[prev] += freq
                indices[prev][j] += 1
            # assuming a symbol sequence "A BC B", if "B C" is merged, increase the frequency of "BC B"
            # however, if the sequence is A BC BC, skip this step because the count of "BC BC" will be incremented by the previous code block
            if i < len(word)-1 and word[i+1] != new_pair:
                nex = word[i:i+2]
                stats[nex] += freq
                indices[nex][j] += 1
            i += 1


In [10]:
def get_pair_statistics(vocab):
    """Count frequency of all symbol pairs, and create index"""

    # data structure of pair frequencies
    stats = defaultdict(int)

    #index from pairs to words
    indices = defaultdict(lambda: defaultdict(int))

    for i, (word, freq) in enumerate(vocab):
        prev_char = word[0]
        for char in word[1:]:
            stats[prev_char, char] += freq
            indices[prev_char, char][i] += 1
            prev_char = char

    return stats, indices


def replace_pair(pair, vocab, indices):
    """Replace all occurrences of a symbol pair ('A', 'B') with a new symbol 'AB'"""
    first, second = pair
    pair_str = ''.join(pair)
    pair_str = pair_str.replace('\\','\\\\')
    changes = []
    pattern = re.compile(r'(?<!\S)' + re.escape(first + ' ' + second) + r'(?!\S)')
    if sys.version_info < (3, 0):
        iterator = indices[pair].iteritems()
    else:
        iterator = indices[pair].items()
    for j, freq in iterator:
        if freq < 1:
            continue
        word, freq = vocab[j]
        new_word = ' '.join(word)
        new_word = pattern.sub(pair_str, new_word)
        new_word = tuple(new_word.split(' '))

        vocab[j] = (new_word, freq)
        changes.append((j, new_word, word, freq))

    return changes

def prune_stats(stats, big_stats, threshold):
    """Prune statistics dict for efficiency of max()
    The frequency of a symbol pair never increases, so pruning is generally safe
    (until we the most frequent pair is less frequent than a pair we previously pruned)
    big_stats keeps full statistics for when we need to access pruned items
    """
    for item,freq in list(stats.items()):
        if freq < threshold:
            del stats[item]
            if freq < 0:
                big_stats[item] += freq
            else:
                big_stats[item] = freq


def learn_bpe(infile, outfile, num_symbols, min_frequency=2, verbose=False, is_dict=False, total_symbols=False):
    """Learn num_symbols BPE operations from vocabulary, and write to outfile.
    """

    # version 0.2 changes the handling of the end-of-word token ('</w>');
    # version numbering allows bckward compatibility
    outfile.write('#version: 0.2\n')

    vocab = get_vocabulary(infile, is_dict)
    vocab = dict([(tuple(x[:-1])+(x[-1]+'</w>',) ,y) for (x,y) in vocab.items()])
    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

    stats, indices = get_pair_statistics(sorted_vocab)
    big_stats = copy.deepcopy(stats)

    if total_symbols:
        uniq_char_internal = set()
        uniq_char_final = set()
        for word in vocab:
            for char in word[:-1]:
                uniq_char_internal.add(char)
            uniq_char_final.add(word[-1])
        sys.stderr.write('Number of word-internal characters: {0}\n'.format(len(uniq_char_internal)))
        sys.stderr.write('Number of word-final characters: {0}\n'.format(len(uniq_char_final)))
        sys.stderr.write('Reducing number of merge operations by {0}\n'.format(len(uniq_char_internal) + len(uniq_char_final)))
        num_symbols -= len(uniq_char_internal) + len(uniq_char_final)
        for i in uniq_char_internal:
            vocab_index2units2freq[i] = 0

    # threshold is inspired by Zipfian assumption, but should only affect speed
    threshold = max(stats.values()) / 10
    for i in range(num_symbols):
        if stats:
            most_frequent = max(stats, key=lambda x: (stats[x], x))

        # we probably missed the best pair because of pruning; go back to full statistics
        if not stats or (i and stats[most_frequent] < threshold):
            prune_stats(stats, big_stats, threshold)
            stats = copy.deepcopy(big_stats)
            most_frequent = max(stats, key=lambda x: (stats[x], x))
            # threshold is inspired by Zipfian assumption, but should only affect speed
            threshold = stats[most_frequent] * i/(i+10000.0)
            prune_stats(stats, big_stats, threshold)

        if stats[most_frequent] < min_frequency:
            sys.stderr.write('no pair has frequency >= {0}. Stopping\n'.format(min_frequency))
            break
        
        #essential
        s1 = most_frequent[0].replace('</w>','')
        s2 = most_frequent[1].replace('</w>','')
        
        vocab_index2units2freq[s1+s2] = stats[most_frequent]
        
        if verbose:
            sys.stderr.write('pair {0}: {1} {2} -> {1}{2} (frequency {3})\n'.format(i, most_frequent[0], most_frequent[1], stats[most_frequent]))
        outfile.write('{0} {1}\n'.format(*most_frequent))
        freq_codes.append(most_frequent)
        changes = replace_pair(most_frequent, sorted_vocab, indices)
        update_pair_statistics(most_frequent, changes, stats, indices)
        stats[most_frequent] = 0
        if not i % 100:
            prune_stats(stats, big_stats, threshold)

In [11]:
vocab_index2units2freq = {}
seq = (df_seq[0].values)
output = open('drug_codes_drug_bank_freq_100.txt', 'w+')
learn_bpe(seq, output, 30000, min_frequency=20, verbose=True, is_dict=False, total_symbols=True)

Number of word-internal characters: 54
Number of word-final characters: 15
Reducing number of merge operations by 69
pair 0: = C -> =C (frequency 35095)
pair 1: O ) -> O) (frequency 24715)
pair 2: C C -> CC (frequency 22096)
pair 3: [ C -> [C (frequency 18298)
pair 4: [C @ -> [C@ (frequency 18243)
pair 5: C ( -> C( (frequency 17635)
pair 6: H ] -> H] (frequency 17560)
pair 7: C =C -> C=C (frequency 15971)
pair 8: = O) -> =O) (frequency 13470)
pair 9: [C@ @ -> [C@@ (frequency 8814)
pair 10: C ) -> C) (frequency 8323)
pair 11: C 1 -> C1 (frequency 7783)
pair 12: C( =O) -> C(=O) (frequency 7721)
pair 13: H] ( -> H]( (frequency 7369)
pair 14: [ H] -> [H] (frequency 6646)
pair 15: =C C=C -> =CC=C (frequency 6103)
pair 16: ( O) -> (O) (frequency 4611)
pair 17: [H] ) -> [H]) (frequency 4369)
pair 18: C 2 -> C2 (frequency 4363)
pair 19: ( [H]) -> ([H]) (frequency 4110)
pair 20: 1 ) -> 1) (frequency 3996)
pair 21: [C@ H]( -> [C@H]( (frequency 3968)
pair 22: = N -> =N (frequency 3946)
pair 23: =

pair 205: CC CN -> CCCN (frequency 190)
pair 206: [C@@ H]3 -> [C@@H]3 (frequency 187)
pair 207: N (C) -> N(C) (frequency 185)
pair 208: O 1) -> O1) (frequency 184)
pair 209: CC1 =C( -> CC1=C( (frequency 183)
pair 210: C2=CC=C C=C2 -> C2=CC=CC=C2 (frequency 180)
pair 211: CC1=CC=C C=C1) -> CC1=CC=CC=C1) (frequency 178)
pair 212: C2 ) -> C2) (frequency 176)
pair 213: C(=O) CC -> C(=O)CC (frequency 175)
pair 214: =C 2 -> =C2 (frequency 175)
pair 215: =CC=C 1 -> =CC=C1 (frequency 174)
pair 216: NC(=O) [C@@H]( -> NC(=O)[C@@H]( (frequency 173)
pair 217: C(F)(F) F) -> C(F)(F)F) (frequency 170)
pair 218: C=C C=C -> C=CC=C (frequency 169)
pair 219: C [C@@H]( -> C[C@@H]( (frequency 168)
pair 220: CC2 =CC=C -> CC2=CC=C (frequency 167)
pair 221: C(=O)N CC -> C(=O)NCC (frequency 165)
pair 222: [H][C@]1 2 -> [H][C@]12 (frequency 164)
pair 223: C2=CC=C C=C2) -> C2=CC=CC=C2) (frequency 162)
pair 224: C(N) =O) -> C(N)=O) (frequency 160)
pair 225: C1=CC=C (O) -> C1=CC=C(O) (frequency 159)
pair 226: (=O)

pair 399: C(=N ) -> C(=N) (frequency 76)
pair 400: C l -> Cl (frequency 76)
pair 401: = O. -> =O. (frequency 76)
pair 402: [C@@]([H])(O) [C@]([H])(O) -> [C@@]([H])(O)[C@]([H])(O) (frequency 75)
pair 403: C1=CC=C( F) -> C1=CC=C(F) (frequency 75)
pair 404: =N 1) -> =N1) (frequency 75)
pair 405: CN ( -> CN( (frequency 74)
pair 406: CCCN C(N)=N) -> CCCNC(N)=N) (frequency 74)
pair 407: C=C (O) -> C=C(O) (frequency 74)
pair 408: C1=CC2=C( C=C1) -> C1=CC2=C(C=C1) (frequency 74)
pair 409: C([O-]) =O</w> -> C([O-])=O</w> (frequency 74)
pair 410: C(=O)N 1</w> -> C(=O)N1</w> (frequency 74)
pair 411: [C@@]([H]) (C) -> [C@@]([H])(C) (frequency 73)
pair 412: O C</w> -> OC</w> (frequency 73)
pair 413: CCCC N) -> CCCCN) (frequency 73)
pair 414: (CC1=CC=C C=C1) -> (CC1=CC=CC=C1) (frequency 73)
pair 415: [C@] (O)( -> [C@](O)( (frequency 72)
pair 416: [C@@] 4 -> [C@@]4 (frequency 72)
pair 417: CO C1=CC=C -> COC1=CC=C (frequency 72)
pair 418: =N 2) -> =N2) (frequency 72)
pair 419: [H] N( -> [H]N( (frequen

pair 578: (O) C=C2) -> (O)C=C2) (frequency 48)
pair 579: [C@@]2([H]) [C@]3([H]) -> [C@@]2([H])[C@]3([H]) (frequency 47)
pair 580: [C@@]1(C)CC [C@@]1([H])[C@@]2([H])CC -> [C@@]1(C)CC[C@@]1([H])[C@@]2([H])CC (frequency 47)
pair 581: S(O) (=O)=O</w> -> S(O)(=O)=O</w> (frequency 47)
pair 582: N) N=CN=C23) -> N)N=CN=C23) (frequency 47)
pair 583: N C(=N) -> NC(=N) (frequency 47)
pair 584: C=C( C=C2) -> C=C(C=C2) (frequency 47)
pair 585: C=C( C=C1) -> C=C(C=C1) (frequency 47)
pair 586: C1=C S -> C1=CS (frequency 47)
pair 587: C(O) =C1 -> C(O)=C1 (frequency 47)
pair 588: (F) (F) -> (F)(F) (frequency 47)
pair 589: (=O) N -> (=O)N (frequency 47)
pair 590: [C@]([H]) (O -> [C@]([H])(O (frequency 46)
pair 591: [ A -> [A (frequency 46)
pair 592: N=C N( -> N=CN( (frequency 46)
pair 593: I) C( -> I)C( (frequency 46)
pair 594: CCCC C2) -> CCCCC2) (frequency 46)
pair 595: CCC 4=C -> CCC4=C (frequency 46)
pair 596: C3 =O) -> C3=O) (frequency 46)
pair 597: (CC (O)=O) -> (CC(O)=O) (frequency 46)
pair 598: 

pair 774: C1=CC( =CC=C1 -> C1=CC(=CC=C1 (frequency 33)
pair 775: C1=C C(=N -> C1=CC(=N (frequency 33)
pair 776: C(O)=O) C(=O) -> C(O)=O)C(=O) (frequency 33)
pair 777: C(O) ( -> C(O)( (frequency 33)
pair 778: C(=O)N([H]) [C@@H]( -> C(=O)N([H])[C@@H]( (frequency 33)
pair 779: [O - -> [O- (frequency 32)
pair 780: [H][C@@]12 C -> [H][C@@]12C (frequency 32)
pair 781: [ 2 -> [2 (frequency 32)
pair 782: F C(F)(F) -> FC(F)(F) (frequency 32)
pair 783: CO C1=C -> COC1=C (frequency 32)
pair 784: CN (C -> CN(C (frequency 32)
pair 785: CC(=O) N[C@@H]( -> CC(=O)N[C@@H]( (frequency 32)
pair 786: C=C( OC) -> C=C(OC) (frequency 32)
pair 787: C1=CC=C2 N -> C1=CC=C2N (frequency 32)
pair 788: C1=C2N=CN( [C@@H]3 -> C1=C2N=CN([C@@H]3 (frequency 32)
pair 789: C(C) =N -> C(C)=N (frequency 32)
pair 790: C(=O)N[C@@H]( CCC(O)=O) -> C(=O)N[C@@H](CCC(O)=O) (frequency 32)
pair 791: C(=O)N( C=C1) -> C(=O)N(C=C1) (frequency 32)
pair 792: C(=O)N C1=O) -> C(=O)NC1=O) (frequency 32)
pair 793: C(=O)N 1) -> C(=O)N1) (freq

pair 934: [C@@]1([H]) O -> [C@@]1([H])O (frequency 25)
pair 935: [2 H]) -> [2H]) (frequency 25)
pair 936: OC[C@H]1O[C@H]([C@H](O) [C@@H]1O) -> OC[C@H]1O[C@H]([C@H](O)[C@@H]1O) (frequency 25)
pair 937: OC(=O) CCCC -> OC(=O)CCCC (frequency 25)
pair 938: O [C@@H](C) -> O[C@@H](C) (frequency 25)
pair 939: NC(=O)[C@H]( N) -> NC(=O)[C@H](N) (frequency 25)
pair 940: NC(=O)[C@H]( CCCNC(N)=N) -> NC(=O)[C@H](CCCNC(N)=N) (frequency 25)
pair 941: NC(=O)[C@H]( CCCCN) -> NC(=O)[C@H](CCCCN) (frequency 25)
pair 942: C\C=C/ C\C=C/ -> C\C=C/C\C=C/ (frequency 25)
pair 943: CN 3 -> CN3 (frequency 25)
pair 944: CCCN 2 -> CCCN2 (frequency 25)
pair 945: CC1=CC=C C=C1 -> CC1=CC=CC=C1 (frequency 25)
pair 946: CC C1 -> CCC1 (frequency 25)
pair 947: C=C C4=C3 -> C=CC4=C3 (frequency 25)
pair 948: C2=C1 N -> C2=C1N (frequency 25)
pair 949: C2=C( N -> C2=C(N (frequency 25)
pair 950: C2=C N -> C2=CN (frequency 25)
pair 951: C2 CC2) -> C2CC2) (frequency 25)
pair 952: C(\ C) -> C(\C) (frequency 25)
pair 953: C(O)=C 2)

pair 1095: C(=O) CO</w> -> C(=O)CO</w> (frequency 21)
pair 1096: 2 =C1 -> 2=C1 (frequency 21)
pair 1097: (CC2=CC=C C=C2) -> (CC2=CC=CC=C2) (frequency 21)
pair 1098: \C(CCC [C@]12C) -> \C(CCC[C@]12C) (frequency 20)
pair 1099: \C(CCC[C@]12C) =C\ -> \C(CCC[C@]12C)=C\ (frequency 20)
pair 1100: \C(CCC[C@]12C)=C\ C=C1 -> \C(CCC[C@]12C)=C\C=C1 (frequency 20)
pair 1101: [O H -> [OH (frequency 20)
pair 1102: [H][C@@]12 C[C@@H](C) -> [H][C@@]12C[C@@H](C) (frequency 20)
pair 1103: [C@]([H]) (CC(C)C) -> [C@]([H])(CC(C)C) (frequency 20)
pair 1104: [C@H]( OC(=O) -> [C@H](OC(=O) (frequency 20)
pair 1105: [C@@]1([H]) CC -> [C@@]1([H])CC (frequency 20)
pair 1106: [C@@] (C)(O) -> [C@@](C)(O) (frequency 20)
pair 1107: P(O)(=O)O P(O)(=O)O -> P(O)(=O)OP(O)(=O)O (frequency 20)
pair 1108: O[C@H](CO)[C@@H](O)[C@H](O) [C@H]2O) -> O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O) (frequency 20)
pair 1109: OCC (O)=O) -> OCC(O)=O) (frequency 20)
pair 1110: O [C@@]([H])(O -> O[C@@]([H])(O (frequency 20)
pair 1111: N2C=N C3=C2 -